# LU 분해


행렬 $A$가 주어졌을 때, 다음과 같은 행렬 $L$과 $U$를 찾습니다.

- $LU = A$
- $L$는 주 대각선 항목이 1과 같은 하 삼각형 행렬입니다.
- $U$는 상 삼각형 행렬입니다.

$$
\begin{matrix}
A = \left[ \begin{array}{cccc} * & * & * & * \\ * & * & * & * \\ * & * & * & * \\ * & * & * & *  \end{array}\right]\hspace{1cm}
L = \left[ \begin{array}{cccc} 1 & 0 & 0 & 0 \\ * & 1 & 0 & 0 \\ * & * & 1 & 0 \\ * & * & * & 1 \end{array}\right]\hspace{1cm}
U = \left[ \begin{array}{cccc} * & * & * & * \\ 0 & * & * & * \\ 0 & 0 & * & * \\ 0 & 0 & 0 & *  \end{array}\right]\hspace{1cm}
\end{matrix}
$$

알려진 행렬 $A$에서 $L$과 $U$를 계산하는 문제를 다루기 전에 이러한 인수분해가 왜 유용한지 알아보겠습니다. L$과 $U$를 구하여 $A=LU$가 되고, $AX=B$ 시스템을 풀고 싶다고 가정해 보겠습니다. 이 문제를 쓰는 또 다른 방법은 $LUX=B$입니다. 그런 다음 $UX=Y$라고 하여 미지의 또 다른 $Y$를 정의하고, 단일 시스템 $AX=B$를 다음 두 시스템으로 교환할 수 있습니다.

$$
\begin{matrix}
UX & = & Y\\
LY & = & B 
\end{matrix}
$$

실제로 방정식의 수가 두 배로 늘어난 것은 사실이지만, 우리가 가지고 있는 두 시스템은 삼각형이며 뒤로(또는 앞으로) 치환하면 쉽게 풀 수 있습니다.

## 예제 1: LU 분해를 활용

아래와 같은 방정식을 풀고자 합니다.

$$
\left[ \begin{array}{ccc} 3 & -1 & -2 \\ 6 & -1 & 0  \\ -3 & 5 & 20  \end{array}\right]X = 
\left[ \begin{array}{c} -4 \\ -8 \\ 6  \end{array}\right]\hspace{1cm}
$$

여기서 $X$는 $3 \times 1$ 벡터입니다. 또한 $L$과 $U$도 계산했다고 가정합니다.

$$
L = \left[ \begin{array}{ccc} 1 & 0 & 0 \\ 2 & 1 & 0  \\ -1 & 4 & 1  \end{array}\right] \hspace{2cm} 
U = \left[ \begin{array}{ccc} 3 & -1 & -2 \\ 0 & 1 & 4  \\ 0 & 0 & 2  \end{array}\right] 
$$


In [1]:
import numpy as np
import scipy.linalg as sla
## Use Python to check for yourself that LU = A.

이제 $UX=Y$ 및 $LY = B$ 시스템을 작성해 보겠습니다. 정확성을 행렬 표기법은 잠시 제쳐두고 $X$의 항목에는 $x_1$, $x_2$, $x_3$ 변수를, $Y$의 항목에는 $y_1$, $y_2$, $y_3$ 변수를 사용합니다.

$$
\begin{matrix}
x_1 \hspace{2.1cm}& = & y_1\\
2x_1 + x_2 \hspace{1.1cm}& = & y_2\\
-x_1 + 4x_2 +x_3 & = & y_3 \\
\\
3y_1 - y_2 - 2y_3 & = & -4\\
y_2 + 4y_3 & = & -8\\
2y_3 & = & 6 
\end{matrix}
$$

이제 해결책은 치환의 문제입니다. 마지막 방정식은 $y_3$을 알려줍니다. 거기서부터 거꾸로 작업하여 $y_2$와 $y_1$을 찾습니다. 그런 다음 첫 번째 방정식부터 시작하여 아래로 내려가면서 비슷한 방식으로 $x$ 값을 구합니다.

## 기본 행렬

제거를 통해 LU 인수분해를 구성하는 방법을 이해하려면 **기본 행렬**이라는 특수 행렬을 곱셈하여 제거 단계를 수행할 수 있다는 것을 알면 도움이 됩니다.  기본 행렬은 호환 가능한 모양의 행렬에 **row_scale** 또는 **row_add** 연산을 적용한 결과입니다. (*행 재배열은 피벗 위치에서 0이 발생하는 경우에만 필요하다는 점을 기억하세요. 행 스왑은 곧 다루겠습니다.)

In [2]:
def row_swap(A,k,l):
    m = A.shape[0]
    n = A.shape[1]
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def row_scale(A,k,scale):
    m = A.shape[0]
    n = A.shape[1]
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def row_add(A,k,l,scale):
    m = A.shape[0]
    n = A.shape[1]
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B

In [3]:
I = np.eye(4)
E = row_add(I,1,2,-3)
print(I,'\n')
print(E)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0. -3.  1.  0.]
 [ 0.  0.  0.  1.]]


우리가 얻는 $E$는 $I$의 세 번째 행에 $I$의 첫 번째 행의 -3배를 더한 결과입니다.  기본 행렬 $E$의 흥미로운 속성은 다른 행렬 $A$에 $E$를 곱하면 그 결과가 $A$에 동일한 행 연산을 적용하여 얻을 수 있는 행렬이 된다는 것입니다.

In [4]:
A=np.array([[1,2,0,-1],[-1,1,-1,4],[2,13,-4,9],[-2,5,-3,13]])
print(A,'\n')
print(E@A)

[[ 1  2  0 -1]
 [-1  1 -1  4]
 [ 2 13 -4  9]
 [-2  5 -3 13]] 

[[ 1.  2.  0. -1.]
 [-1.  1. -1.  4.]
 [ 5. 10. -1. -3.]
 [-2.  5. -3. 13.]]


## 예제 2:  Finding an LU factorization

이제 일련의 기본 행렬 $E_1$, $E_2$, $E_3$,...을 $A$에 적용하여 제거를 수행할 수 있습니다. 위의 행렬에서 어떻게 작동하는지 살펴봅시다.

In [5]:
A=np.array([[1,2,0,-1],[-1,1,-1,4],[2,13,-4,9],[-2,5,-3,13]])
I = np.eye(4)
E1 = row_add(I,0,1,1)
E2 = row_add(I,0,2,-2)
E3 = row_add(I,0,3,2)
print(E3@E2@E1@A,'\n')
E4 = row_add(I,1,2,-3)
E5 = row_add(I,1,3,-3)
print(E5@E4@E3@E2@E1@A)

[[ 1.  2.  0. -1.]
 [ 0.  3. -1.  3.]
 [ 0.  9. -4. 11.]
 [ 0.  9. -3. 11.]] 

[[ 1.  2.  0. -1.]
 [ 0.  3. -1.  3.]
 [ 0.  0. -1.  2.]
 [ 0.  0.  0.  2.]]


**row_add**를 사용하여 적절한 공간에 $0$을 만들었으므로 이제 $U$ 계수가 생겼습니다. 행렬 곱셈을 기호로 작성하면 다음과 같이 표시됩니다.

$$
\begin{matrix}
E_5E_4E_3E_2E_1A = U
\end{matrix}
$$

곱셈 순서는 변경할 수 없습니다. $E_1$이 가장 먼저 $A$를 곱하고, 그다음에 $E_2$를 곱하는 식으로 곱해야 합니다. 이제 역행렬의 속성에 따라 기호를 약간 조작해 보겠습니다.

$$
\begin{matrix}
A &=& (E_5E_4E_3E_2E_1)^{-1}U  \\
A &=& E_1^{-1}E_2^{-1}E_3^{-1}E_4^{-1}E_5^{-1}U  
\end{matrix}
$$

$L = E_1^{-1}E_2^{-1}E_3^{-1}E_4^{-1}E_5^{-1}$가 되어야 합니다.  이 역행렬의 곱이 $L$이라는 정확한 형태를 갖는다는 사실은 전혀 명확하지 않습니다.  다음 두 가지를 관찰해 봅시다.

- 역행렬의 각 행렬은 단순한 하한 삼각형 구조를 가집니다. 사실 $E_3^{-1}$ 행렬도 기본 행렬입니다. $E_3$로 표현되는 행렬 연산을 취소하는 행렬이 바로 기본 행렬입니다! $E_3$를 곱하면 첫 번째 행의 2배를 마지막 행에 더하게 됩니다. $E_3^{-1}$로 곱하면 마지막 행에 첫 번째 행의 -2배를 더합니다.

In [6]:
print(E3,'\n')
print(sla.inv(E3))

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [2. 0. 0. 1.]] 

[[ 1. -0. -0.  0.]
 [ 0.  1. -0.  0.]
 [ 0.  0.  1.  0.]
 [-2.  0.  0.  1.]]


- 두 개의 하위 삼각형 행렬을 곱하면 하위 삼각형 행렬이 만들어집니다.

In [7]:
L1 = np.array([[1,0,0,0],[-1,1,0,0],[2,3,1,0],[-2,3,0,1]])
L2 = np.array([[1,0,0,0],[2,1,0,0],[-5,4,1,0],[4,4,1,1]])
print(L1,'\n')
print(L2,'\n')
print(L1@L2)

[[ 1  0  0  0]
 [-1  1  0  0]
 [ 2  3  1  0]
 [-2  3  0  1]] 

[[ 1  0  0  0]
 [ 2  1  0  0]
 [-5  4  1  0]
 [ 4  4  1  1]] 

[[1 0 0 0]
 [1 1 0 0]
 [3 7 1 0]
 [8 7 1 1]]


이 두 가지 사실을 종합하면 $E_1^{-1}E_2^{-1}E_3^{-1}E_4^{-1}E_5^{-1}$ 행렬이 $L$ 인자가 되기 위한 올바른 구조를 가지고 있음을 알 수 있습니다. 더 편리한 점은 이러한 역행렬을 함께 곱할 때 아래쪽 삼각형 부분의 0이 아닌 항목은 변하지 않는다는 것입니다.

In [8]:
print(sla.inv(E5),'\n')
print(sla.inv(E4)@sla.inv(E5),'\n')
print(sla.inv(E3)@sla.inv(E4)@sla.inv(E5))

[[ 1. -0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0. -0.  1.  0.]
 [ 0.  3.  0.  1.]] 

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 3. 1. 0.]
 [0. 3. 0. 1.]] 

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  3.  1.  0.]
 [-2.  3.  0.  1.]]


$E_3^{-1}E_4^{-1}E_5^{-1}$의 0이 아닌 하위 삼각형 항목은 $E_3^{-1}$, $E_4^{-1}$ 및 $E_5^{-1}$의 해당 항목과 동일합니다.  즉, $L$의 항목은 $\texttt{RowAdd}$의 적용에 사용된 배율에 -1을 곱한 값일 뿐입니다.  이제 이러한 기본 행렬이 어떻게 결합하여 $L$을 생성하는지 이해했으므로 실제로 행렬을 구성할 필요는 없습니다. 행 연산을 할 때 배율을 추적하여 $L$을 계산하면 됩니다.  

In [9]:
L = np.array([[1,0,0,0],[-1,1,0,0],[2,3,1,-0],[-2,3,0,1]])
U = np.array([[1,2,0,-1],[0,3,-1,3],[0,0,-1,2],[0,0,0,2]])
print("L:",'\n',L,'\n',sep='')
print("U:",'\n',U,'\n',sep='')
print("LU:",'\n',L@U,sep='')

L:
[[ 1  0  0  0]
 [-1  1  0  0]
 [ 2  3  1  0]
 [-2  3  0  1]]

U:
[[ 1  2  0 -1]
 [ 0  3 -1  3]
 [ 0  0 -1  2]
 [ 0  0  0  2]]

LU:
[[ 1  2  0 -1]
 [-1  1 -1  4]
 [ 2 13 -4  9]
 [-2  5 -3 13]]
